In [2]:
# !pip install python-dotenv
from dotenv import load_dotenv
import os
# os.getcwd()

load_dotenv()
openai_api_key = os.environ.get("OPENAI_API_KEY")

In [ ]:
# 6.1
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter()

loader = UnstructuredFileLoader("./files/chapter_one.docx")

loader.load_and_split(text_splitter=splitter)

In [ ]:
# 6.2
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)


loader = UnstructuredFileLoader("./files/chapter_one.docx")

In [4]:
import os
os.getcwd()

'd:\\Library\\Code\\NomadCoders_FullstackGPT'

In [5]:
# 6.4
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import Chroma
from langchain.storage import LocalFileStore

cache_dir = LocalFileStore("./.cache/")


splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("./files/chapter_one.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

# 존재하면 캐시된 임베딩을 사용하고, 그렇지 않으면 새로운 임베딩을 계산
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

# vectorstore = Chroma.from_documents(docs, embeddings)
vectorstore = Chroma.from_documents(docs, cached_embeddings)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\AI.Public\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\AI.Public\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


In [ ]:
results = vectorstore.similarity_search("where does winston live")

results # 문서가 떨어짐

In [7]:
# 6.6
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA

llm = ChatOpenAI()

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("./files/chapter_one.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

# RetrievalQA를 사용하여 질문을 요청하고 답변을 얻음
chain = RetrievalQA.from_chain_type(
    llm=llm,
    # chain_type="stuff",
    # chain_type="refine",
    # chain_type="mapreduce",
    chain_type="map_rerank",
    retriever=vectorstore.as_retriever(),
)

# chain.run("Where does Winston live?")
chain.run("Describe Victory Mansions")

d:\.venv\lib\site-packages\langchain\chains\llm.py:349: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


'Victory Mansions is a rooftop where you can see four buildings: Ministry of Truth, Ministry of Peace, Ministry of Love, and Ministry of Plenty. It is dwarfed by the surrounding architecture and is home to the four Ministries. It is a place where the entire apparatus of government was divided.'

In [ ]:
# 6.8
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(
    temperature=0.1,
)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("./files/chapter_one.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

# input = single string(question), output = list of docs
retriver = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}",
        ),
        ("human", "{question}"),
    ]
)

# chain = retriver | prompt | llm

chain = (
    {
        "context": retriver,
        "question": RunnablePassthrough(),
        # "extra": RunnablePassthrough(),
    }
    | prompt
    | llm
)

chain.invoke("Describe Victory Mansions")

In [ ]:
# 6.9 MapReduce
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

llm = ChatOpenAI(
    temperature=0.1,
)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("./files/chapter_one.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriever = vectorstore.as_retriever()

# list of docs
# for doc in list | prompt | llm

# for response in list of llms response | put them together
map_doc_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Use the following portion of a long document to see if any of the text is relevant to answer the question. Return any relevant text verbatim. If there is no relevant text, return : ''
            -------
            {context}
            """,
        ),
        ("human", "{question}"),
    ]
)

map_doc_chain = map_doc_prompt | llm

# final doc | prompt | llm
# 하나의 문서로 만들어주는 것
def map_docs(inputs):
    documents = inputs["documents"]
    question = inputs["question"]
    # for document in documents:
    #     result = map_doc_chain.invoke(
    #         {"context": doc.page_content, "question": question}
    #     ).content
    #     results.append(result)
    # result = "\n\n".join(result)
    # return result
    return "\n\n".join(
        map_doc_chain.invoke(
            {"context": doc.page_content, "question": question}
        ).content # page_content는 문서의 내용
        for doc in documents # list comprehension
    )


map_chain = {
    "documents": retriever,
    "question": RunnablePassthrough(),
} | RunnableLambda(map_docs) # function을 불러올 수 있는 기능

# final doc | prompt | llm
final_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Given the following extracted parts of a long document and a question, create a final answer. 
            If you don't know the answer, just say that you don't know. Don't try to make up an answer.
            ------
            {context}
            """,
        ),
        ("human", "{question}"),
    ]
)

chain = {"context": map_chain, "question": RunnablePassthrough()} | final_prompt | llm

chain.invoke("How many ministries are mentioned")d

In [14]:
# Homework
# 1. Stuff Documents
# 2. ConversationBufferMemory
# 3. QA
# 3.1. Aaronson은 유죄인가요?
# 3.2. 그가 테이블에 어떤 메시지를 썼나요?
# 3.3. Julia는 누구인가요?
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough
from langchain.memory import ConversationSummaryBufferMemory

llm = ChatOpenAI(
    temperature=0.1,
)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("./files/document.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

# input = single string(question), output = list of docs
retriver = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}",
        ),
        # MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,
)

def load_memory(_):
    return memory.load_memory_variables({})["history"]
    # return memory.load_memory_variables({}).get("history", [])

# chain = retriver | prompt | llm
# chain = RunnablePassthrough.assign(history=load_memory) | final_prompt | chat
chain = (
    {
        "context": retriver,
        "question": RunnablePassthrough(),
        # "history": RunnablePassthrough(history=load_memory),
        # "extra": RunnablePassthrough(),
    }
    | prompt
    | llm
)

# def invoke_chain(question):
#     result = chain.invoke({"question": question})
#     memory.save_context(
#         {"input": question},
#         {"output": result.content},
#     )

chain.invoke("Is Aaronson guilty?")
# invoke_chain("Is Aaronson guilty?")

AIMessage(content='According to the text, Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with.')

In [15]:
chain.invoke("What message did he write on the table?")
# invoke_chain("What message did he write on the table?")

AIMessage(content='He wrote "2+2=5" on the table.')

In [16]:
chain.invoke("Who is Julia?")
# invoke_chain("Who is Julia?")

AIMessage(content='Julia is a character in the text who is significant to the protagonist.')